# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here:

In [1]:
from urllib.request import urlretrieve
from os.path import isfile
from tqdm import tqdm

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile('train.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Train Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/train.p',
            'train.p',
            pbar.hook)

if not isfile('test.p'):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Test Dataset') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/udacity-sdc/datasets/german_traffic_sign_benchmark/test.p',
            'test.p',
            pbar.hook)

print('Training and Test data downloaded.')

Training and Test data downloaded.


## Overview

Here are the steps you'll take to build the network:

1. Load the training data.
2. Preprocess the data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate the final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [2]:
import pickle
import numpy as np
import math

# Fix error with TF and Keras
import tensorflow as tf
tf.python.control_flow_ops = tf

print('Modules loaded.')

Modules loaded.


## Load the Data

Start by importing the data from the pickle file.

In [3]:
with open('train.p', 'rb') as f:
    data = pickle.load(f)

# TODO: Load the feature data to the variable X_train


# TODO: Load the label data to the variable y_train

data.keys()

dict_keys(['sizes', 'features', 'labels', 'coords'])

In [4]:
X_train = data['features']
y_train = data['labels']

In [5]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert np.array_equal(X_train, data['features']), 'X_train not set to data[\'features\'].'
assert np.array_equal(y_train, data['labels']), 'y_train not set to data[\'labels\'].'
print('Tests passed.')

Tests passed.


## Preprocess the Data

1. Shuffle the data
2. Normalize the features using Min-Max scaling between -0.5 and 0.5
3. One-Hot Encode the labels

### Shuffle the data
Hint: You can use the [scikit-learn shuffle](http://scikit-learn.org/stable/modules/generated/sklearn.utils.shuffle.html) function to shuffle the data.

In [6]:
X_train.shape

(39209, 32, 32, 3)

In [7]:
# TODO: Shuffle the data
from sklearn.utils import shuffle

print(X_train[0,:,:,0], "\n")
X_train, y_train = shuffle(X_train, y_train, random_state=42)
print(X_train[0,:,:,0])

[[ 75  74  83 ...,  67  65  66]
 [ 82  80  87 ...,  73  75  79]
 [ 79  84  91 ...,  72  74  78]
 ..., 
 [126 118 108 ...,  96  99  99]
 [ 90  91  77 ..., 104 107  95]
 [ 85  93  69 ..., 101  98  90]] 

[[105  95  78 ..., 139 139 180]
 [106  92  89 ...,  95  68  93]
 [114  97 100 ...,  90  67  74]
 ..., 
 [ 94  92  84 ...,  94 100  97]
 [ 98 102 108 ..., 103 114 101]
 [115 122 121 ..., 109  83  81]]


In [8]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert X_train.shape == data['features'].shape, 'X_train has changed shape. The shape shouldn\'t change when shuffling.'
assert y_train.shape == data['labels'].shape, 'y_train has changed shape. The shape shouldn\'t change when shuffling.'
assert not np.array_equal(X_train, data['features']), 'X_train not shuffled.'
assert not np.array_equal(y_train, data['labels']), 'y_train not shuffled.'
print('Tests passed.')

Tests passed.


### Normalize the features
Hint: You solved this in [TensorFlow lab](https://github.com/udacity/CarND-TensorFlow-Lab/blob/master/lab.ipynb) Problem 1.

In [9]:
# TODO: Normalize the data features to the variable X_normalized

def normalize(x, a, b):
    return a + (x-x.min())*(b-a)/(x.max() - x.min())

a, b = -0.5, 0.5
X_normalized = normalize(X_train, a, b)

In [10]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert math.isclose(np.min(X_normalized), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_normalized), 0.5, abs_tol=1e-5), 'The range of the training data is: {} to {}.  It must be -0.5 to 0.5'.format(np.min(X_normalized), np.max(X_normalized))
print('Tests passed.')

Tests passed.


### One-Hot Encode the labels
Hint: You can use the [scikit-learn LabelBinarizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) function to one-hot encode the labels.

In [11]:
y_train.shape

(39209,)

In [12]:
y_train[:10]

array([26, 15, 13, 10, 15, 12, 25,  2, 11, 35], dtype=uint8)

In [13]:
len(np.unique(y_train))

43

In [14]:
# TODO: One Hot encode the labels to the variable y_one_hot
from sklearn.preprocessing import LabelBinarizer


lb = LabelBinarizer()
y_one_hot = lb.fit_transform(y_train)

In [15]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
import collections

assert y_one_hot.shape == (39209, 43), 'y_one_hot is not the correct shape.  It\'s {}, it should be (39209, 43)'.format(y_one_hot.shape)
assert next((False for y in y_one_hot if collections.Counter(y) != {0: 42, 1: 1}), True), 'y_one_hot not one-hot encoded.'
print('Tests passed.')

Tests passed.


## Keras Sequential Model
```python
from keras.models import Sequential

# Create the Sequential model
model = Sequential()
```
The `keras.models.Sequential` class is a wrapper for the neural network model. Just like many of the class models in scikit-learn, it provides common functions like `fit()`, `evaluate()`, and `compile()`.  We'll cover these functions as we get to them.  Let's start looking at the layers of the model.

## Keras Layer
A Keras layer is just like a neural network layer.  It can be fully connected, max pool, activation, etc.  You can add a layer to the model using the model's `add()` function.  For example, a simple model would look like this:
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

# Create the Sequential model
model = Sequential()

# 1st Layer - Add a flatten layer
model.add(Flatten(input_shape=(32, 32, 3)))

# 2nd Layer - Add a fully connected layer
model.add(Dense(100))

# 3rd Layer - Add a ReLU activation layer
model.add(Activation('relu'))

# 4th Layer - Add a fully connected layer
model.add(Dense(60))

# 5th Layer - Add a ReLU activation layer
model.add(Activation('relu'))
```
Keras will automatically infer the shape of all layers after the first layer.  This means you only have to set the input dimensions for the first layer.

The first layer from above, `model.add(Flatten(input_shape=(32, 32, 3)))`, sets the input dimension to (32, 32, 3) and output dimension to (3072=32\*32\*3).  The second layer takes in the output of the first layer and sets the output dimension to (100).  This chain of passing output to the next layer continues until the last layer, which is the output of the model.

## Build a Multi-Layer Feedforward Network

Build a multi-layer feedforward neural network to classify the traffic sign images.

1. Set the first layer to a `Flatten` layer with the `input_shape` set to (32, 32, 3)
2. Set the second layer to `Dense` layer width to 128 output. 
3. Use a ReLU activation function after the second layer.
4. Set the output layer width to 43, since there are 43 classes in the dataset.
5. Use a softmax activation function after the output layer.

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [18]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

model = Sequential()
# TODO: Build a Multi-layer feedforward neural network with Keras here.

model.add(Flatten(input_shape=(32,32,3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [19]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax

def check_layers(layers, true_layers):
    assert len(true_layers) != 0, 'No layers found'
    for layer_i in range(len(layers)):
        assert isinstance(true_layers[layer_i], layers[layer_i]), 'Layer {} is not a {} layer'.format(layer_i+1, layers[layer_i].__name__)
    assert len(true_layers) == len(layers), '{} layers found, should be {} layers'.format(len(true_layers), len(layers))

check_layers([Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[1].output_shape == (None, 128), 'Second layer output is wrong, it should be (128)'
assert model.layers[2].activation == relu, 'Third layer not a relu activation layer'
assert model.layers[3].output_shape == (None, 43), 'Fourth layer output is wrong, it should be (43)'
assert model.layers[4].activation == softmax, 'Fifth layer not a softmax activation layer'
print('Tests passed.')

Tests passed.


## Training a Sequential Model
You built a multi-layer neural network in Keras, now let's look at training a neural network.
```python
from keras.models import Sequential
from keras.layers.core import Dense, Activation

model = Sequential()
...

# Configures the learning process and metrics
model.compile('sgd', 'mean_squared_error', ['accuracy'])

# Train the model
# History is a record of training loss and metrics
history = model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)

# Calculate test score
test_score = model.evaluate(x_test_data, Y_test_data)
```
The code above configures, trains, and tests the model.  The line `model.compile('sgd', 'mean_squared_error', ['accuracy'])` configures the model's optimizer to `'sgd'`(stochastic gradient descent), the loss to `'mean_squared_error'`, and the metric to `'accuracy'`.  

You can find more optimizers [here](https://keras.io/optimizers/), loss functions [here](https://keras.io/objectives/#available-objectives), and more metrics [here](https://keras.io/metrics/#available-metrics).

To train the model, use the `fit()` function as shown in `model.fit(x_train_data, Y_train_data, batch_size=128, nb_epoch=2, validation_split=0.2)`.  The `validation_split` parameter will split a percentage of the training dataset to be used to validate the model.  The model can be further tested with the test dataset using the `evaluation()` function as shown in the last line.

## Train the Network

1. Compile the network using adam optimizer and categorical_crossentropy loss function.
2. Train the network for ten epochs and validate with 20% of the training data.

In [26]:
# TODO: Compile and train the model here.

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_normalized, y_one_hot, batch_size=64, nb_epoch=10, validation_split=0.2)

Train on 31367 samples, validate on 7842 samples
Epoch 1/10
31367/31367 [==============================] - 6s - loss: 0.2987 - acc: 0.9147 - val_loss: 0.2710 - val_acc: 0.9285
Epoch 2/10
31367/31367 [==============================] - 6s - loss: 0.2695 - acc: 0.9246 - val_loss: 0.5372 - val_acc: 0.8537
Epoch 3/10
31367/31367 [==============================] - 6s - loss: 0.2437 - acc: 0.9306 - val_loss: 0.3395 - val_acc: 0.9049
Epoch 4/10
31367/31367 [==============================] - 6s - loss: 0.2398 - acc: 0.9313 - val_loss: 0.2982 - val_acc: 0.9142
Epoch 5/10
31367/31367 [==============================] - 6s - loss: 0.2225 - acc: 0.9381 - val_loss: 0.2705 - val_acc: 0.9250
Epoch 6/10
31367/31367 [==============================] - 6s - loss: 0.2137 - acc: 0.9378 - val_loss: 0.2627 - val_acc: 0.9268
Epoch 7/10
31367/31367 [==============================] - 6s - loss: 0.2170 - acc: 0.9367 - val_loss: 0.3339 - val_acc: 0.9049
Epoch 8/10
31367/31367 [==============================] - 6s -

In [27]:
history.history

{'acc': [0.91471929097459115,
  0.92463416967066037,
  0.93059584914081683,
  0.93126534255746485,
  0.93808779927949759,
  0.93776899289061755,
  0.93671693180921367,
  0.9465999298644947,
  0.94892721650141865,
  0.94688685561258645],
 'loss': [0.29865410380740259,
  0.26949892446308948,
  0.24370030368813278,
  0.23976175240492029,
  0.22254949686220707,
  0.21367465456247625,
  0.21697418470326749,
  0.18699653246769712,
  0.17921396549543472,
  0.18082331854410744],
 'val_acc': [0.92846212690200647,
  0.85373629167109721,
  0.90487120635531992,
  0.91418005600172592,
  0.92501912766711747,
  0.92680438651445107,
  0.90487120634011853,
  0.90410609527742092,
  0.89734761541943497,
  0.94465697515500313],
 'val_loss': [0.27101611643383561,
  0.53721054283309666,
  0.33949158617167774,
  0.29815372541543023,
  0.27050565335556609,
  0.26271984621871036,
  0.33389853738349057,
  0.32987897470498445,
  0.36908366683337557,
  0.21302473821739737]}

### Analyzing the results in two executions changing the hyperparameter batch_size:

with batch_size = 128 we got this

```python
history.history =
{'acc': [0.53062135365287733,
  0.76956674212131249,
  0.84241400198040006,
  0.86584627156308247,
  0.89463448847704929,
  0.90827940191921441,
  0.91599451653011121,
  0.91953326744667963,
  0.92670641120218111,
  0.93257244875377321],
 'loss': [1.812161593721926,
  0.88465736501547609,
  0.61362845317267045,
  0.5090975846658079,
  0.4104346794819404,
  0.35715664474450776,
  0.32126913589594652,
  0.29845630031973469,
  0.27446618816517898,
  0.25617076475691641],
 'val_acc': [0.56477939313359782,
  0.72545269059453865,
  0.78168834474919191,
  0.84264218304054495,
  0.85564906902381332,
  0.84799795961294877,
  0.89033409848987854,
  0.87107880645733471,
  0.85360877316571471,
  0.82708492722325222],
 'val_loss': [1.8080863077036735,
  0.90002324416363189,
  0.68619735015651695,
  0.53700261842198171,
  0.48678915048703586,
  0.52226086376242964,
  0.36606352255831442,
  0.4326373497705866,
  0.48974334736832303,
  0.72965114487886606]}
```

but with batch_size = 64 we got

```python
history.history = 
{'acc': [0.91471929097459115,
  0.92463416967066037,
  0.93059584914081683,
  0.93126534255746485,
  0.93808779927949759,
  0.93776899289061755,
  0.93671693180921367,
  0.9465999298644947,
  0.94892721650141865,
  0.94688685561258645],
 'loss': [0.29865410380740259,
  0.26949892446308948,
  0.24370030368813278,
  0.23976175240492029,
  0.22254949686220707,
  0.21367465456247625,
  0.21697418470326749,
  0.18699653246769712,
  0.17921396549543472,
  0.18082331854410744],
 'val_acc': [0.92846212690200647,
  0.85373629167109721,
  0.90487120635531992,
  0.91418005600172592,
  0.92501912766711747,
  0.92680438651445107,
  0.90487120634011853,
  0.90410609527742092,
  0.89734761541943497,
  0.94465697515500313],
 'val_loss': [0.27101611643383561,
  0.53721054283309666,
  0.33949158617167774,
  0.29815372541543023,
  0.27050565335556609,
  0.26271984621871036,
  0.33389853738349057,
  0.32987897470498445,
  0.36908366683337557,
  0.21302473821739737]}
```

batch_size = 128 -> val_acc = 0.82708492722325222
batch_size = 64 -> val_acc = 0.94465697515500313

** batch_size is a crucial hyperparameter!!!! **

In [28]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.optimizers import Adam

assert model.loss == 'categorical_crossentropy', 'Not using categorical_crossentropy loss function'
assert isinstance(model.optimizer, Adam), 'Not using adam optimizer'
assert len(history.history['acc']) == 10, 'You\'re using {} epochs when you need to use 10 epochs.'.format(len(history.history['acc']))

assert history.history['acc'][-1] > 0.92, 'The training accuracy was: %.3f. It shoud be greater than 0.92' % history.history['acc'][-1]
assert history.history['val_acc'][-1] > 0.85, 'The validation accuracy is: %.3f. It shoud be greater than 0.85' % history.history['val_acc'][-1]
print('Tests passed.')

Tests passed.


## Convolutions
1. Re-construct the previous network
2. Add a [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
3. Add a ReLU activation after the convolutional layer.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

In [31]:
# TODO: Re-construct the network and add a convolutional layer before the flatten layer.
from keras.layers.convolutional import Convolution2D

model = Sequential()
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='valid', input_shape=(32, 32,3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [32]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D

check_layers([Convolution2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)

assert model.layers[0].input_shape == (None, 32, 32, 3), 'First layer input shape is wrong, it should be (32, 32, 3)'
assert model.layers[0].nb_filter == 32, 'Wrong number of filters, it should be 32'
assert model.layers[0].nb_col == model.layers[0].nb_row == 3, 'Kernel size is wrong, it should be a 3x3'
assert model.layers[0].border_mode == 'valid', 'Wrong padding, it should be valid'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 48s - loss: 1.1387 - acc: 0.7092 - val_loss: 0.4402 - val_acc: 0.8920
Epoch 2/2
31367/31367 [==============================] - 48s - loss: 0.2902 - acc: 0.9296 - val_loss: 0.2485 - val_acc: 0.9406
Tests passed.


## Pooling
1. Re-construct the network
2. Add a 2x2 [max pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

In [33]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='valid', input_shape=(32, 32,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [34]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[1].pool_size == (2, 2), 'Second layer must be a max pool layer with pool size of 2x2'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 30s - loss: 1.4670 - acc: 0.6131 - val_loss: 0.6196 - val_acc: 0.8331
Epoch 2/2
31367/31367 [==============================] - 30s - loss: 0.4056 - acc: 0.9002 - val_loss: 0.3127 - val_acc: 0.9200
Tests passed.


## Dropout
1. Re-construct the network
2. Add a [dropout](https://keras.io/layers/core/#dropout) layer after the pooling layer. Set the dropout rate to 50%.

In [37]:
# TODO: Re-construct the network and add dropout after the pooling layer.
from keras.layers.core import Dropout

model = Sequential()
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='valid', input_shape=(32, 32,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))

In [38]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

check_layers([Convolution2D, MaxPooling2D, Dropout, Activation, Flatten, Dense, Activation, Dense, Activation], model.layers)
assert model.layers[2].p == 0.5, 'Third layer should be a Dropout of 50%'

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=128, nb_epoch=2, validation_split=0.2)
assert(history.history['val_acc'][-1] > 0.91), "The validation accuracy is: %.3f.  It should be greater than 0.91" % history.history['val_acc'][-1]
print('Tests passed.')

Train on 31367 samples, validate on 7842 samples
Epoch 1/2
31367/31367 [==============================] - 33s - loss: 1.5317 - acc: 0.5979 - val_loss: 0.6790 - val_acc: 0.8137
Epoch 2/2
31367/31367 [==============================] - 32s - loss: 0.4903 - acc: 0.8722 - val_loss: 0.3654 - val_acc: 0.9044


AssertionError: The validation accuracy is: 0.904.  It should be greater than 0.91

Let's be honest: I don't give a shit about this test. All the parameters are fixed: "add this layer here", "this one after this one", "with this value", etc. If I follow the instructions carefully there is no way I can pass the test rather than relying in the randomness of the training (initialization and dropout conexions) and repeat it many times until it gets an accuracy over the selected threshold. So let's forget about this result and see what can I achieve in the next section where I can really use my skills.

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [39]:
# TODO: Build a model

# TODO: Compile and train the model

model = Sequential()

model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='valid', input_shape=(32, 32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(nb_filter=64, nb_row=3, nb_col=3, border_mode='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(43))
model.add(Activation('softmax'))


model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [40]:
history = model.fit(X_normalized, y_one_hot, batch_size=64, nb_epoch=10, validation_split=0.2)

Train on 31367 samples, validate on 7842 samples
Epoch 1/10
31367/31367 [==============================] - 56s - loss: 2.3598 - acc: 0.3251 - val_loss: 0.9584 - val_acc: 0.7299
Epoch 2/10
31367/31367 [==============================] - 57s - loss: 1.0525 - acc: 0.6601 - val_loss: 0.3753 - val_acc: 0.9101
Epoch 3/10
31367/31367 [==============================] - 57s - loss: 0.6842 - acc: 0.7770 - val_loss: 0.2221 - val_acc: 0.9471
Epoch 4/10
31367/31367 [==============================] - 57s - loss: 0.5293 - acc: 0.8268 - val_loss: 0.1611 - val_acc: 0.9671
Epoch 5/10
31367/31367 [==============================] - 57s - loss: 0.4313 - acc: 0.8601 - val_loss: 0.1263 - val_acc: 0.9690
Epoch 6/10
31367/31367 [==============================] - 57s - loss: 0.3517 - acc: 0.8853 - val_loss: 0.0891 - val_acc: 0.9819
Epoch 7/10
31367/31367 [==============================] - 57s - loss: 0.3161 - acc: 0.8987 - val_loss: 0.0783 - val_acc: 0.9832
Epoch 8/10
31367/31367 [==============================]

**Best Validation Accuracy:** (fill in here)

val_acc: 0.9872

These results show that the generalization capabilities of my model are great. However, the achieved training acuracy is low enough (for this dataset) to consider building a more complex architecture.

In [52]:
# Let's save the trained model
json_string = model.to_json()

with open("final_trained_model.json", "w") as fout:
    fout.writelines(json_string)

# We can not use the following line as h5py was not installed before doing this exercise. We can install it, of course
# but we will have to traing again. Dammit!! Next time I should check these things before.
#model.save_weights("final_trained_model.hdf5")

# But there is an alternative:

import pickle

with open("final_trained_model.pkl", "wb") as fout:
    weights = model.get_weights()
    pickle.dump(weights, fout, protocol=pickle.HIGHEST_PROTOCOL) 
    # The advantage of HIGHEST_PROTOCOL is that files get smaller. This makes unpickling sometimes much faster.

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the [`metrics_names`](https://keras.io/models/model/) property to get the labels.

In [53]:
# TODO: Load test data
    
# TODO: Preprocess data & one-hot encode the labels

# TODO: Evaluate model on test data

with open('test.p', 'rb') as f:
    test_data = pickle.load(f)

X_test = test_data['features']
y_test = test_data['labels']

a, b = -0.5, 0.5
X_test_normalized = normalize(X_test, a, b)

lb = LabelBinarizer()
y_test_one_hot = lb.fit_transform(y_test)

In [55]:
res = model.evaluate(X_test_normalized, y_test_one_hot)
print(res)
print(model.metrics_names)

12630/12630 [==============================] - 7s     
[0.16229501442625374, 0.95558194779066086]
['loss', 'acc']


**Test Accuracy:** (fill in here)

test_acc: 0.95558194779066086

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.